# Eigenfaces: Dogs and Cats

Based on the article: L. Sirovich and M. Kirby; "Low-dimensional procedure for the characterization of human faces."

## Imports

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

## Data collection

## Data normalization and preparation

In [ ]:
# transformation to a uniform format (e.g. in grayscale, resolution)
# reduce differences in lighting or animal orientation

## Converting images to matrices

In [ ]:
# convert images into pixel value matrices
# each image will become a data vector, where each pixel is one value of the vector

## Calculation of the average image

In [ ]:
# calculate the average image from a set of photos by summing the pixel values of each image and dividing by the number of images

## Calculation of the deviation from the average image

In [ ]:
# subtract the mean image from each image to obtain the deviation matrices

## Construction of the covariance matrix

In [ ]:
# use the deviations to build a covariance matrix
# this matrix will represent how pixels differ from the mean relative to each other

## Calculation of eigenvalues and vectors

In [ ]:
# calculate the eigenvalues and eigenvectors of the covariance matrix
# the eigenvectors will be "eigenpictures" (eigenanimals), which are the main components of the image

## Image reconstruction

In [ ]:
# each image can now be expressed as a linear combination of these eigenanimals
# this allows you to reduce the amount of data by storing only the coefficients of this combination

## Classification

In [ ]:
# classification using eigenfaces coefficients

---

### PCA metodą Sirovicha i Kirby'ego

In [ ]:
import os
import cv2
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def load_images_from_folder(folder, size=(100, 100)):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Konwersja do skali szarości
            images.append(img.flatten())
    return images


# Ładowanie obrazów kotów i psów
cat_images = load_images_from_folder('cat', size=(100, 100))
dog_images = load_images_from_folder('dog', size=(100, 100))

# Łączenie obrazów kotów i psów w jeden zbiór danych
all_images = np.array(cat_images + dog_images)

# Normalizacja danych
mean_image = np.mean(all_images, axis=0)
centered_images = all_images - mean_image

# Przeprowadzenie PCA
pca = PCA(n_components=50)  # Liczba głównych składowych
pca.fit(centered_images)

eigenfaces = pca.components_.reshape((50, 100, 100))

# Wizualizacja eigenfaces
fig, axes = plt.subplots(5, 10, figsize=(15, 8))
for i, ax in enumerate(axes.flat):
    ax.imshow(eigenfaces[i], cmap='gray')
    ax.axis('off')
plt.show()

# Rekonstrukcja obrazów
reconstructed_images = pca.inverse_transform(pca.transform(centered_images))
reconstructed_images = reconstructed_images.reshape((-1, 100, 100))

# Wizualizacja oryginalnych i zrekonstruowanych obrazów
fig, axes = plt.subplots(2, 10, figsize=(15, 4))
for i in range(10):
    # Oryginalne obrazy
    ax = fig.add_subplot(2, 10, i + 1)
    ax.imshow(all_images[i].reshape(100, 100), cmap='gray')
    ax.axis('off')

    # Zrekonstruowane obrazy
    ax = fig.add_subplot(2, 10, i + 11)
    ax.imshow(reconstructed_images[i], cmap='gray')
    ax.axis('off')
plt.show()

## Przygotowanie danych i obliczenie eigenfaces

In [ ]:
import os
import cv2
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Funkcja do ładowania obrazów i przekształcania ich na wektory
def load_images_from_folder(folder, size=(100, 100)):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Konwersja do skali szarości
            images.append(img.flatten())
    return images


# Ładowanie obrazów kotów i psów
cat_images = load_images_from_folder('cat', size=(100, 100))
dog_images = load_images_from_folder('dog', size=(100, 100))

# Łączenie obrazów kotów i psów w jeden zbiór danych
all_images = np.array(cat_images + dog_images)
labels = np.array([0] * len(cat_images) + [1] * len(dog_images))  # 0 = kot, 1 = pies

# Normalizacja danych
mean_image = np.mean(all_images, axis=0)
centered_images = all_images - mean_image

# Przeprowadzenie PCA
n_components = 50
pca = PCA(n_components=n_components)
pca.fit(centered_images)

# Ekstrakcja współczynników eigenfaces
eigenfaces = pca.components_.reshape((n_components, 100, 100))
transformed_images = pca.transform(centered_images)

## Ekstrakcja współczynników eigenfaces
Współczynniki eigenfaces to wyniki transformacji PCA (transformed_images), które mogą być używane jako cechy do klasyfikacji.

## Klasyfikacja
Użyjemy współczynników eigenfaces do trenowania modelu klasyfikacji SVM.

In [ ]:
# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(transformed_images, labels, test_size=0.2, random_state=42)

# Trenowanie modelu SVM
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(X_train, y_train)

# Predykcja na zbiorze testowym
y_pred = svm.predict(X_test)

# Ocena modelu
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność: {accuracy * 100:.2f}%")
print("Raport klasyfikacji:\n", classification_report(y_test, y_pred, target_names=['Kot', 'Pies']))

## Podsumowanie kodu:
- Ładowanie i przetwarzanie obrazów: Ładowanie obrazów kotów i psów, zmniejszanie rozmiaru do 100x100 pikseli i konwersja do skali szarości.
- Przygotowanie danych: Obliczenie średniego obrazu i przekształcenie obrazów na macierz odchyleń.
- PCA: Przeprowadzenie PCA, aby uzyskać główne składowe (eigenfaces) i przekształcenie obrazów na współczynniki eigenfaces.
- Klasyfikacja: Podział danych na zbiór treningowy i testowy, trenowanie modelu SVM na współczynnikach eigenfaces, predykcja i ocena modelu.